In [340]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split


import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [341]:
#load data
AC = np.load('C:\Github_LIU\SII2025\process_data\classifier\AC.npy')
AD = np.load('C:\Github_LIU\SII2025\process_data\classifier\AD.npy')
BC = np.load('C:\Github_LIU\SII2025\process_data\classifier\BC.npy')
BD = np.load('C:\Github_LIU\SII2025\process_data\classifier\BD.npy')
#split data into training and testing and validation
X = np.concatenate((AC, AD, BC, BD), axis=0)
AC_labels = [0] * len(AC)
AD_labels = [1] * len(AD)
BC_labels = [2] * len(BC)
BD_labels = [3] * len(BD)
y = np.concatenate((AC_labels, AD_labels, BC_labels, BD_labels), axis=0)
#归一化
def min_max_normalize(data):
    return (data - np.min(data, axis=0)) / (np.max(data, axis=0) - np.min(data, axis=0))
X = min_max_normalize(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, X_val.shape,y_train.shape, y_test.shape, y_val.shape)
batch_size = 20
train_data = torch.utils.data.TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_data = torch.utils.data.TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val))
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)
test_data = torch.utils.data.TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test))
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)


(150, 301, 12) (48, 301, 12) (38, 301, 12) (150,) (48,) (38,)


In [342]:
#define hyperparameters
sequence_len = 301
input_len = 12
hidden_size = 128
num_layers = 2
num_classes = 10
num_epochs = 1000
learning_rate = 0.01



In [343]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [344]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, num_classes)
        #self.relu = nn.ReLU()

    def forward(self, x):
        hidden_state = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        cell_state = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (hidden_state, cell_state))
        out = self.output_layer(out[:, -1, :])
        return out

In [345]:
model = LSTM(input_len, hidden_size, num_layers, num_classes).to(device)
print(model)

LSTM(
  (lstm): LSTM(12, 128, num_layers=2, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=10, bias=True)
)


In [346]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [347]:
def train(num_epochs, model, train_loader, loss_function):
    total_step = len(train_loader)
    
    for epoch in range(num_epochs):
        for batch, (images, labels) in enumerate(train_loader):
            images = images.reshape(-1, sequence_len, input_len).to(device)
            labels = labels.long().to(device)
            
            outputs = model(images)
            loss = loss_function(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (batch+1) % 1 == 0:
                print(f"Epoch [{epoch+1}; Batch {batch+1}/{total_step}]; Loss: {loss.item():.4f}")
    


In [348]:
train(num_epochs, model, train_loader,loss_function)

Epoch [1; Batch 2/8]; Loss: 2.0103
Epoch [1; Batch 4/8]; Loss: 1.6352
Epoch [1; Batch 6/8]; Loss: 1.4723
Epoch [1; Batch 8/8]; Loss: 1.3732
Epoch [2; Batch 2/8]; Loss: 1.5168
Epoch [2; Batch 4/8]; Loss: 1.4764
Epoch [2; Batch 6/8]; Loss: 1.6002
Epoch [2; Batch 8/8]; Loss: 1.5946
Epoch [3; Batch 2/8]; Loss: 1.5043
Epoch [3; Batch 4/8]; Loss: 1.4102
Epoch [3; Batch 6/8]; Loss: 1.3802
Epoch [3; Batch 8/8]; Loss: 1.3835
Epoch [4; Batch 2/8]; Loss: 1.4550
Epoch [4; Batch 4/8]; Loss: 1.4515
Epoch [4; Batch 6/8]; Loss: 1.4369
Epoch [4; Batch 8/8]; Loss: 1.4224
Epoch [5; Batch 2/8]; Loss: 1.4385
Epoch [5; Batch 4/8]; Loss: 1.5021
Epoch [5; Batch 6/8]; Loss: 1.4053
Epoch [5; Batch 8/8]; Loss: 1.3763
Epoch [6; Batch 2/8]; Loss: 1.4507
Epoch [6; Batch 4/8]; Loss: 1.3523
Epoch [6; Batch 6/8]; Loss: 1.4299
Epoch [6; Batch 8/8]; Loss: 1.3636
Epoch [7; Batch 2/8]; Loss: 1.4141
Epoch [7; Batch 4/8]; Loss: 1.5000
Epoch [7; Batch 6/8]; Loss: 1.5026
Epoch [7; Batch 8/8]; Loss: 1.4177
Epoch [8; Batch 2/8]